<a href="https://colab.research.google.com/github/maheshbabu-r/BIG_DATA_PySpark/blob/main/Reviews__Analysis_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz  # supress download output use -q

!tar -zxvf spark-3.1.2-bin-hadoop3.2.tgz | grep "something" 2>/dev/null #Suppress tar output ---| grep "something" 2>/dev/null--- add after file_name"

!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext

#from pyspark.sql.types import StructType,StructField, StringType, IntegerType

# or import like this
from pyspark.sql.functions import *

from pyspark.sql import functions as F

from pyspark.sql.types import *

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.appName("Analysis Reviews").getOrCreate()
print(spark.sparkContext.appName)

In [ ]:
!wget -q https://raw.githubusercontent.com/maheshbabu-r/BIG_DATA/main/Hadoop%20Datasets/senticnet4.txt

!wget -q https://github.com/maheshbabu-r/BIG_DATA/blob/main/Hadoop%20Datasets/amazondataset.rar?raw=true

# rename file for better understanding
!mv amazondataset.rar?raw=true amazondataset.rar

!unrar x /content/amazondataset.rar

In [ ]:
df=spark.read.csv(header=True,inferSchema=True,path="/content/amazondataset.csv")

df.printSchema()

df.show(5)

In [ ]:
df1=spark.read.option("sep", "\t").csv("/content/senticnet4.txt")

df1.printSchema()

df1.show(3)

In [ ]:
schema = StructType([ \
    StructField("_c0",StringType(),True), \
    StructField("_c1",StringType(),True),\
    StructField("_c2",FloatType(), True) \
  ])

In [ ]:
df1=spark.read.option("sep", "\t").csv("/content/senticnet4.txt",schema=schema)

df1.printSchema()

df1.show(3)

In [ ]:
df1 = df1.withColumnRenamed("_c0","words") \
    .withColumnRenamed("_c1","sentiment") \
    .withColumnRenamed("_c2","value")
df1.printSchema()

In [ ]:
df1.show(3)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.boxplot(x=df1.toPandas()["value"].values.tolist())
plt.show()

In [ ]:
x=df1.toPandas()["words"].values.tolist()
y=df1.toPandas()["value"].values.tolist()
ratings=dict(zip(x,y))

In [ ]:
def sum_of_list(l):
  total = 0
  for val in l:
    total = total + val
  return total

def reviews_value(reviews):
  y=''.join(e.lower() for e in reviews if (e.isalnum() or e.isspace()))
  y=[x for x in y.split(" ")]
  z=[ratings.get(x) for x in y]
  final=[(0 if x==None else x) for x in z]
  return sum_of_list(final)

In [ ]:
texts=df.toPandas()["reviewstext"].values.tolist()
values=[reviews_value(x) for x in texts]

In [ ]:
def add_labels(indx):
    return values[indx-1] # since row num begins from 1
labels_udf = udf(add_labels, DoubleType())

In [ ]:
df.createOrReplaceTempView('A')
B = spark.sql('select row_number() over(order by "mahesh" ) as num, * from A')

B.show(5)


In [ ]:
new_df = B.withColumn('Rating', labels_udf('num'))
new_df.show()